![imagenes](logo.png)

# Reducción de dimensionalidad

Es el proceso de transformar un conjunto de datos de alta dimensión (con muchas variables o características) a uno de menor dimensión conservando la mayor cantidad posible de información relevante. Se usa para:

- Visualización (reducir a 2D o 3D),

- Eliminación de ruido o redundancia,

- Mejora en rendimiento de algoritmos,

- Prevención de sobreajuste.

# Descomposición en Valores Singulares (SVD)

Imagina que eres un escultor y tienes frente a ti un bloque de mármol. Primero, lo giras cuidadosamente para que quede perfectamente alineado con tu espacio de trabajo. Luego comienzas a esculpir, pero no con la misma intensidad en todas las direcciones; algunas partes las trabajas más profundamente que otras, como si aplicaras un escalado selectivo. Finalmente, una vez terminada la escultura, la giras para colocarla en su posición final, lista para exhibirse


## ¿Qué es la SVD?
Para una matriz $A \in \mathbb{R}^{m \times n}$, la descomposición en valores singulares (SVD) es:
$$
A = U \Sigma V^T
$$
donde:
- $U \in \mathbb{R}^{m \times m}$ y $V \in \mathbb{R}^{n \times n}$ son matrices ortogonales,
- $\Sigma$ es una matriz diagonal (o rectangular con ceros) con valores no negativos en la diagonal (los valores singulares).

En términos de nuestro escultor:
1. **Giras** el bloque para alinearlo (eso hace $V^T$),
2. **Esculpes** con diferentes profundidades (eso hace $\Sigma$),
3. **Colocas** el resultado en su orientación final (eso hace $U$).

Así actúa el SVD: cambio de base → escalado → nueva orientación.

## Geometría de la transformación
Aplicar $A$ a un vector es como:
- cambiar de base con $V^T$, 
- escalar con $\Sigma$,
- rotar con $U$.

Un círculo se transforma en una elipse cuyos ejes principales están dados por las columnas de $U$ y alargadas según los valores singulares.

In [5]:
# Ejemplo numérico de SVD
import numpy as np
A = np.array([[3, 1], [0, 2]])
U, S, VT = np.linalg.svd(A)

print('A=',A)
print('U =\n', U)
print('\nValores singulares =', S)
print('\nV^T =\n', VT)

# Reconstrucción de A
Sigma = np.zeros_like(A, dtype=float)
np.fill_diagonal(Sigma, S)
print('Sigma=',Sigma)
A_reconstructed = U @ Sigma @ VT
print('\nA reconstruida =\n', A_reconstructed)

A= [[3 1]
 [0 2]]
U =
 [[ 0.95709203 -0.28978415]
 [ 0.28978415  0.95709203]]

Valores singulares = [3.25661654 1.84240298]

V^T =
 [[ 0.8816746   0.47185793]
 [-0.47185793  0.8816746 ]]
Sigma= [[3.25661654 0.        ]
 [0.         1.84240298]]

A reconstruida =
 [[3.00000000e+00 1.00000000e+00]
 [1.11022302e-16 2.00000000e+00]]


## Aplicaciones
| Aplicación | Descripción |
|------------|-------------|
| Compresión de imágenes | Se eliminan valores singulares pequeños |
| PCA | PCA es SVD sobre la matriz centrada |
| Resolución de sistemas | La pseudo-inversa usa SVD |
| Filtrado de ruido | Se descartan componentes con bajo peso |


## Aproximación de rango bajo (Teorema de Eckart–Young)

Los valores singulares de la matriz $A$ son los números no negativos $\sigma_i$ tales que $\sigma_i=\sqrt{\lambda_i}$ donde $\lambda_i$ es un valor propio de $A^TA$.
 
> **Teorema de Eckart–Young**. Sea $A\in\mathbb{R}^{m\times n}$ y con descomposición en valores singulares $$A=U\Sigma V^T$$
> 
>Los valores singulares $\sigma_1,\,\sigma_2,...,\sigma_r>0$ determinan cuánto contribuye cada componente (o dirección) a la acción de $A$.
>
> El Teorema de Eckart–Young dice que si deseas aproximar $A$ por otra matriz de rango reducido $A_r$, entonces la mejor elección (en el sentido de error cuadrático mínimo con norma de Frobenius) es:
>$$A_r=U_r\Sigma_rV_r^T$$ 
>donde solo conservas los primeros $r$ valores singulares y los vectores correspondientes. Es decir, estás "tirando a la basura" las componentes menos importantes (las más pequeñas).

De esta manera, tu bloque de mármol aún será rotado y esculpido, pero no usarás todas tus herramientas. Decides concentrarte solo en los $r$ ejes más importantes de acción: los que definen la forma principal:

- Los valores singulares grandes corresponden a herramientas que modifican mucho la forma.

- Los valores pequeños afectan detalles finos que, en caso de apuro, puedes omitir.

Es como si tallaras solo los rasgos principales del rostro y omitieras las arrugas o texturas del cabello. La escultura resultante sigue pareciendo un rostro, aunque con menos detalle.

## Reducción de dimensionalidad con SVD

Otra manera de interpretar el SVD es la siguiente: Sea $A\in\mathbb{R}^{m\times n}$ con rango $\rho$ y $\sigma_i>0$ los valores singulares ordenados de mayor a menor, $u_i\in\mathbb{R}^m$ las columnas de $U$ (recordemos que son ortonormales) y $v_i\in\mathbb{R}^n$ las columnas de $V$ (recordemos que son ortonormales). Entonces 

$$A=\sigma_1u_1v_1^T+\sigma_2u_2v_2^T+...+\sigma_\rho u_\rho v_\rho^T$$

Entonces la mejor aproximación a $A$ con rango $r$ es 

$$A_r=\sigma_1u_1v_1^T+\sigma_2u_2v_2^T+...+\sigma_r u_r v_r^T$$

Aunque $A_r$ vuelve a tener dimensiones $m\times n$, en la expresión anterior está la magia: **$A_r$ es la suma de solo $r$ matrices de rango 1**, cada una de la forma $\sigma_i u_i v_i^T$, y por lo tanto su **rango es exactamente $r$**.

Esto implica que **todas las columnas de $A_r$** (y también sus filas) **viven en un subespacio de dimensión $r$**. En particular:

- Las columnas de $A_r$ están contenidas en el subespacio generado por $\{u_1, u_2, \dots, u_r\} \subseteq \mathbb{R}^m$,
- Las filas de $A_r$ están contenidas en el subespacio generado por $\{v_1, v_2, \dots, v_r\} \subseteq \mathbb{R}^n$.

De esta manera, el SVD nos dice que **toda la acción de $A_r$** se reduce a una combinación óptima de solo $r$ direcciones fundamentales: $r$ verticales y $r$ horizontales.

Este es el corazón del teorema de Eckart–Young: **de todas las matrices de rango $r$**, $A_r$ es la que **más se parece a $A$** en norma de Frobenius (o espectral). Y esa semejanza está construida, literalmente, **sumando capas**.

## Conclusiones
- La SVD descompone una matriz en rotaciones y escalados.
- Es una herramienta central en ciencia de datos, álgebra lineal y machine learning.
- Permite compresión, reducción de ruido y análisis de estructuras internas.
